This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [1]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium chromedriver_autoinstaller

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import boto3

In [3]:
# The file download processing section should be placed here
import chromedriver_autoinstaller
import time, zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
chromedriver_autoinstaller.install() # had to use this instead of chromedriver-binary because version conflict with selenium
driver = webdriver.Chrome()
driver.get(r'https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC')

# akerZd => class name for downloads on google drive, we wait for the download to appear then download
wait = WebDriverWait(driver,10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME,"akerZd"))).click()

time.sleep(3)
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall()
#...........................................................

In [4]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'klOW1O1c1j0rm3khNR7T')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'lZeO6WJdPA7S4u946KJ01zuCzlTcWq2AWB8pteuh')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [5]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [ ]:
# Upload file to minio
upload_file_to_minio('data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data3.json', 'data', 'data-raw/data3.json')